# Method 5: Fine Tuning - Conversational Pairs

In this series we are exploring how to match tone of a sample. My goal is to instruct and tune the LLM to talk like me. I'll use a few podcasts I've been on as examples.

Check out the [full video](link_to_video) overview of this for more context.

For this method we are going to try to reproduce my speaking style based on conversational pairs. This means I'll take my back and forth conversation w/ a podcast host as the training set for a fine tuned model.

In [1]:
import os, json
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
import re

load_dotenv()

True

In [2]:
chat = ChatOpenAI(model='gpt-4', openai_api_key=os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE"))

### Previous work
We did a bunch of work in the previous methods to get my tone description, relevant docs to our sample query and writing examples. We'll load those up here so we don't need to run the code again.

In [3]:
# This is a text file of a bunch of lines that I said
with open("Transcripts/GregLines.txt", 'r') as file:
    greg_lines = file.read()

# This is a description of my tone as determined by the LLM (previous method)
with open("gregs_tone_description.txt", 'r') as file:
    gregs_tone_description = file.read()

# This are specific references to me talking about a previous role that I had
with open("first_job_college_relevant_docs.txt", 'r') as file:
    relevant_docs = file.read()

# These are specific examples of how I talk, similar to the GregLines above
with open("greg_example_writing.txt", 'r') as file:
    writing_examples = file.read()

### Fine Tuning

Now we are going to move onto the fun part, fine tuning. I want to use a open sourced model to save on costs and see how a model that *hasn't* been trained on so much safety does.

To do this I'm going to fine tune and run my model via [Gradient.ai](https://gradient.ai/) who helped sponsor this video. Super easy to get set up and the team has been responsive.

### Step 1: Create Conversational Pair

When you fine tune it's recommended to have a set of validated 'input' and 'output' pairs. This is your training set. I'm going to use my transcripts as the output, but what do use for the input?

I'm going to use the back and forth conversation of the podcasts I've been on. The question or statement that preceeds my response will be in input and my response will be the output.

In [4]:
transcripts = ['Transcripts/transcript_1.txt', 'Transcripts/transcript_2.txt', 'Transcripts/transcript_3.txt']

result = []

for transcript_file in transcripts:
    transcript = open(transcript_file, 'r').read()
    
    # Split up the sections
    sections = re.split(r'(\d{2}:\d{2}:\d{2} [A-Za-z]+:)', transcript)
    sections = [section.strip() for section in sections if section.strip()]  # Removing any empty strings

    # Now, we'll pair up the speaker names with their corresponding content
    paired_sections = [(sections[i], sections[i+1]) for i in range(0, len(sections), 2)]

    # Extracting 'input' and 'output' pairs
    
    for i in range(1, len(paired_sections)):
        if paired_sections[i][0].startswith("00:00:00 Greg:"):
            continue
        if paired_sections[i][0].startswith("00:") and "Greg:" in paired_sections[i][0]:
            input_text = paired_sections[i-1][1]
            output_text = paired_sections[i][1]
            
            # Remove long examples due to context limit
            if len(input_text) > 1000 or len(output_text) > 1000:
                continue
            
            result.append({'input': input_text, 'output': output_text})

Great let's take a look at a sample

In [5]:
result[31:33]

[{'input': 'yeah,',
  'output': 'you can you I mean, My my DMs are open on Twitter, and they will be for the foreseeable future. So, just hit me up there.'},
 {'input': 'Yes. However Right. Right.',
  'output': "The benchmarks all show that open source aren't as good as the closed models. I'm a capitalist at heart, and so I love market pressure. And so I love the fact that there are multiple model providers all battling for consumer value and consumer attention. Because as a consumer myself, I will gladly, love the benefit of them battling for my"}]

Let's see how many data points we have

In [6]:
len(result)

67

Ok, so 67 data points to work with, let's get started

Great, now that we have our input/output pairs, let's transformt them into training data points. You can see what the suggested format is on [Gradient's website](https://docs.gradient.ai/docs/tips-and-tricks)

In [7]:
training_set = []

for pair in result:
    training_set.append({"inputs": f"<s>### Instruction:\n{pair['input']}\n\n### Response:\n{pair['output']}</s>" })

### Fine Tuning

Now we are on to the fine tuning step. We'll use their Nous Hermes 2 model. You can check out their full list of supported models [here](https://docs.gradient.ai/docs/models-1). You'll need to use python 3.10+

In [8]:
from gradientai import Gradient

# Make your Gradient client
gradient = Gradient(access_token=os.getenv("GRADIENT_API_TOKEN", "YourTokenHere"),
                    workspace_id=os.getenv("GRADIENT_WORKSPACE_ID", "YourWorkSpaceIdHere"))

# Get your base model ready. You'll need to grab the model slug from Gradient's website
base_model = gradient.get_base_model(base_model_slug="nous-hermes2")

In [9]:
# Create your new model which you'll stem from the base model
new_model = base_model.create_model_adapter(
    name="My Greg Model - Conversational Prompts"
)

print(f"Created model adapter with id {new_model.id}")

Created model adapter with id d362b9c1-d793-4e0b-bd7a-dcd8002f37b8_model_adapter


Great! Now let's do the cool part of fine tuning

In [10]:
# Training on the training_set we made above
new_model.fine_tune(samples=training_set)

FineTuneResponse(number_of_trainable_tokens=10113, sum_loss=32148.799)

### Final Output

In [11]:
from langchain.llms import GradientLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [12]:
llm = GradientLLM(
    # `ID` listed in `$ gradient model list`
    model=new_model.id,
    # optional: set new credentials, they default to environment variables
    gradient_workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_access_token=os.environ["GRADIENT_API_TOKEN"],
    model_kwargs=dict(max_generated_token_count=228)
)

In [13]:
question = "What was your first job out of college? Did you like it?"

In [16]:
# Finally, let's load it all up into a good prompt for us to use!

template = """
    <s>
    You are a person named Greg Kamradt
    
    Here is background context to use when answering the question
    **Start of relevant information**
    {background_information}
    **End of relevant information**
    
    Here are some examples of how Greg Kamradt talks, mimic the tone you see here
    **Start of examples information**
    {examples}
    **End of examples information**
    
    % ANSWER THIS QUESTION
    {question}
    
    % YOUR RESPONSE:
    </s>
    """

prompt = PromptTemplate(
    input_variables=["background_information", "question", "examples"],
    template=template,
)

final_prompt = prompt.format(
    background_information = relevant_docs,
    examples=writing_examples,
    question = question
)

llm_answer = llm.predict(final_prompt)

print (llm_answer)

Yeah. Yeah. Yeah. It's a good question. I would say that there's kind of 2 Pivotal pivotal moments. My 1st role out of school. So I'm a fresh fresh out of undergrad, very green in the workplace. And I started my career off in corporate FP and A. And that means I was doing finance and budgets for big corporations in their, their departments. Right? I remember getting an Excel spreadsheet. This was back before Google drive or links or anything like that. So it was just a big massive dirty un version controlled spreadsheet. And, there was a list of transactions throughout the entire page and, you know, call it something like a 100,000 transactions or whatever, more rows than I could count, more columns than I could see. And my boss was basically like, Greg, you tell us what tell us X, Y, Z. Like, I forgot what the question was, but they were like, go figure out the answer within the spreadsheet. And
